# Hackathon Immersion Day
**Camilo Valenzuela - Software Engineer - Data Observatory**

Este es un pequeño tutorial mostrando las distintas formas de acceder a los datos que se proveen en la Hackathon.

Se tienen dos tipos de datos:
1. Imagenes tif (SAF) en un Bucket de AWS S3.
2. Datos dentro del DataCube.

# Accediendo al DataCube

Para acceder al DataCube se utiliza el paquete `datacube` (https://github.com/opendatacube/datacube-core) de Open DataCube

In [2]:
import datacube
from odc.ui import DcViewer
from deafrica_plotting import display_map

ModuleNotFoundError: No module named 'deafrica_plotting'

## Crear una conexion al datacube 

Al crear una conexion con el DataCube se genera automaticamente la conexión con los datos dispobiles para la Hackathon

In [ ]:
dc = datacube.Datacube(app="ImmersionDay")

## Dentro del data cube existen distintos productos

In [ ]:
?dc.list_products

In [ ]:
products = dc.list_products()
products[["name", "description"]]

## Para cada producto se tienen varias mediciones

In [ ]:
?dc.list_measurements

In [ ]:
measurements = dc.list_measurements()
measurements.head(20)

In [ ]:
product = "usgs_espa_ls8c1_veg"
measurements.loc[product]

Por ejemplo el producto *usgs_espa_ls8c1_veg* tiene dos índices de vegetación:
- ndvi: Índice de vegetación de diferencia normalizada (Normalized difference vegetation index)
- evi: Enhanced vegetation index

# Visalizar los productos

In [ ]:
?DcViewer

In [ ]:
display_ls8 = [p for p in products.name if "ls8" in p]
DcViewer(dc=dc, 
         time='2013',
         products=display_ls8,
         center=(-33.45, -70.66),
         zoom=7,
        )

## Obteniendo datos del Data Cube
Para este ejemplo usaremos los datos cercanos a la playa Las Docas en Valparaiso.

In [ ]:
#Definimos nuestra zona de interés
lat_range = (-33.14, -33.16)
lon_range = (-71.72, -71.67)

#Y un rango de tiempo
time_range = ("2013-01-01", "2020-01-01")

In [ ]:
display_map(x=lon_range, y=lat_range)

Los datos se descargan mediante `load()`.

In [ ]:
ds = dc.load(product = product,
             longitude = lon_range,
             latitude = lat_range,
             output_crs = "EPSG:32719",
             time = time_range,
             resolution = (-25, 25),
             dask_chunks = {"time": 1}
            )
ds

In [ ]:
ds.ndvi

## Visualizando los datos

Los datos vienen escalados por lo que tenemos que transformarlos previo a cualquier procesamiento 

https://www.usgs.gov/core-science-systems/nli/landsat/landsat-surface-reflectance-derived-spectral-indices?qt-science_support_page_related_con=0#qt-science_support_page_related_con

In [ ]:
scaled_ds = ds.ndvi[:10]*.0001
scaled_ds.plot(col="time", robust=True, col_wrap=5)

In [ ]:
ds_ndvi = ds.ndvi * .0001
means = ds_ndvi.mean(axis=[1,2]).values

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
plt.figure(figsize=(12,5))
plt.title("Mean NVDI")
plt.plot(means)

Algunas preguntas:
- ¿Que hacemos con la nubosidad?
- Podemos utilizar otros datos para mejorar el análisis.

# Objetener datos desde AWS S3 

## Creamos un cliente de s3 

In [ ]:
import boto3
import pandas as pd
import xarray as xr

In [ ]:
s3 = boto3.client("s3")

## Nos conectamos al bucket de la hackathon

Usando boto3 podemos pedir la lista de imagenes que se tienen dentro del Bucket.

In [ ]:
bucket = "hackathon-fach-2020"

In [ ]:
objects = s3.list_objects(Bucket=bucket)
parsed_objects = pd.DataFrame(objects["Contents"])
parsed_objects.sort_values("Key")

In [ ]:
file = "incendios/04082020/Valparaiso_04082020_pan.tif"
file_path = f's3://{bucket}/{file}'
file_path

### Cargamos los datos en un xarray

In [ ]:
# Si se agrega chunks los datos se cargan mediante Dask, si no con Numpy.
da = xr.open_rasterio(file_path, chunks={'x': 500, 'y': 500})
da

In [ ]:
slice_data = da.sel(band=1, y=slice(*lat_range), x = slice(*lon_range))
slice_data

In [ ]:
# Cargando los datos en memoria
slice_data = slice_data.load()
slice_data.plot()

**Para trabajar fuera de jupyterhub descargar los datos y luego cargarlos de manera local**

## Descargando los datos localmente

Para descargar los datos al jupyterhub utilizamos el AWS Cli.

```bash
aws s3 cp s3://<bucket>/<archivo> <path_dentro_de_jupyterhub>
```

In [ ]:
! aws s3 cp 's3://hackathon-fach-2020/incendios/04082020/Valparaiso_04082020_pan.tif' .

La cuenta de jupyterhub tiene un limite de 20GB de disco duro.